## 모델을 내보내는 방법
1. Keras
```Python
saved_model_path = model.save(file_path=model_path, save_format="tf")
```

2. Estimator
```Python
import tensorflow as tf

# 먼저 reciver_fn 정의가 필요.
def serving_input_receiver_fn():
    input_feature = tf.compat.v1.placeholder(dtype=tf.string,
                                             shape=[None, 1],
                                             name="input")
    fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
        features={"input_feature": input_feature})

    return fn

# `model_fn`은 모델을 리턴하는 함수.
estimator = tf.estimator.Estimator(model_fn, "model", params={})
estimator.export_saved_model(export_dir_base="model_dir",
                             serving_input_receiver_fn=serving_input_receiver_fn)
```

이렇게 내보낸 모델 형식 SavedModel은 다음 구조로 돼 있다:
  - `saved_model.pb`: `MetaGraphDef` 형식으로 표현된 추론 그래프.
  - `variables`: 파라미터를 이진 형식으로 만든 파일과 체크포인트.
  - `assets`: 모델 로드에 필요한 각종 파일들.

## 모델 서명(signature)
서명 함수를 정의한다. 이 서명 함수 호출이 모델에게 추론을 요청하는 API다.
이 함수가 어떤 입력과 출력을 받는지 일정한 형식으로 나타나는 '서명'으로 정의한다.
가능한 서명 함수는 `predict`, `classify`, `regress` 세 가지이다. 단일 그래프는 복수의 서명
함수를 지닐 수 있다.

모델이 어떤 서명을 갖고 있는지는 CLI 명령줄 도구 `saved_model_cli`로 확인 가능.

```bash
$ saved_model_cli show --dir <model_dir>
```
The given SavedModel contains the following tag-sets: 'serve'
여기서 `tag-sets`란 이 그래프에 부여된 파악하기 쉬운 이름이다.
단일 모델일지라도 TFLite용, 일반 서버용 등 복수의 그래프가 포함될 수 있다.

```bash
$ saved_model_cli show --dir <model_dir> --tag_set <tag>
```
다음과 같은 결과를 얻었다.
    The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
    SignatureDef key: "__saved_model_init_op"
    SignatureDef key: "serving_default"

`SignatureDef`는 SavedModel 포맷 모델을 빌드할 때 정의된 서명 함수의 입출력을 보여주는 객체다.
 `serving_default`는 특별히 지정하지 않은 경우 기본적으로 서명되는 함수다.

```bash
$ saved_model_cli show --dir <model_dir> --tag_set <tag> --signature_def <sign>
```
결과:
The given SavedModel SignatureDef contains the following input(s):
  inputs['examples'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_examples:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['outputs'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall_11:0
Method name is: tensorflow/serving/predict  # 이 서명에 묶인 추론 함수.

직접 모델을 로드하지 않고도 `saved_model_cli`로 터미널에서 특정 서명으로 모델 추론을 시험해 볼 수
있다:
```bash
$ saved_model_cli run --dir <model_dir> --tag_set <tag> --signature_def <sign> \
    --input_examples <examples>
```
입력 가능한 형식은 모두 세 가지다:
  - `inputs`: np.ndarray
  - `input_exprs`: Python expression
  - `input_examples`: tf.Example

# TFServing 모델 배포
ExampleGen, Trainer, Pusher 등 일련의 파이프라인 구성에 Serving을 편입시키려면
Pusher가 bless된 모델을 저장하는 경로를 Serving이 모델을 가져오는 경로와 동일하게 설정하면 된다.
Serving은 모델을 쌓아두는 경로에서 가장 넘버링이 큰 모델을 불러오도록 돼 있으니, Pusher가 날짜순 등
으로 모델 디렉토리를 저장하도록 설정하면 자동으로 최신 배포가 가능하다.

Serving은 우분투가 아닌 이상 도커 이미지로 받는 것이 제일 편하다.

```bash
# 8500: gRPC용 포트, 8501: REST API용
# source: 이 디렉토리 안에 모델들이 1/, 2/ 등 모여 있어야 한다.
# target: `$(MODEL_BASE_PATH)/$(MODEL_NAME)`결합이 돼야 한다.
# ARM 아키텍쳐 미지원이라 커뮤니티 지원 사용
docker run -p 8501:8501 -p 8500:8500 \
           --mount type=bind,source=$(pwd)/models,target=/models/complaints \
           -e MODEL_NAME=complaints \
           -e MODEL_BASE_PATH=/models \
           -t emacski/tensorflow-serving
```


## 다중 모델 구성
아래 설정을 파일로 저장 후 `doker --model_config_file=<config_file>`로 도커 기동 시 인수로 지정

```
model_config_list {
  config {
    name: 'my_model'
    base_path: '/models/my_model'
    model_platform: 'tensorflow'
  }
  config {
    name: 'your_model'
    base_path: '/models/your_model'
    model_platform: 'tensorflow'
    model_version_policy: {
      # 다른 옵션(all) 등도 지정 가능.
      specific: {
        versions: 123456
        versions: 654321
      }
    }
    # 아래 버전별 라벨 지정은 선택 사항이다.
    version_labels {
      key: 'stable'
      value: 123456
    }
    version_labels {
      key: 'dev'
      value: 654321
    }
  }
}
```

## 모델 서버로 요청

### REST API

In [14]:
import os
import requests
import json
import base64

import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [6]:
def _bytes_feature(value: str):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value.encode()])
    )


def _float_feature(value: float):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value])
    )


def __int64_feature(value: int):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value])
    )

In [4]:
sample = {
    "product": "XXX",
    "sub_product": "XXX",
    "issue": "XXX",
    "sub_issue": "XXX",
    "state": "NY",
    "zip_code": 63512,
    "company": "XXX",
    "company_response": "XXX",
    "timely_response": "XXX",
    "consumer_disputed": 1,
    "consumer_complaint_narrative": "XXX",
}

In [5]:
def create_example(data: dict):
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                "product": _bytes_feature(str(data["product"])),
                "sub_product": _bytes_feature(str(data["sub_product"])),
                "issue": _bytes_feature(str(data["issue"])),
                "sub_issue": _bytes_feature(str(data["sub_issue"])),
                "state": _bytes_feature(str(data["state"])),
                "zip_code": __int64_feature(int(data["zip_code"])),
                "company": _bytes_feature(str(data["company"])),
                "company_response": _bytes_feature(str(data["company_response"])),
                "timely_response": _bytes_feature(str(data["timely_response"])),
                "consumer_disputed": _float_feature(float(data["consumer_disputed"])),
                "consumer_complaint_narrative": _bytes_feature(str(data["consumer_complaint_narrative"])),
            }
        )
    )

example = create_example(sample)

NameError: name 'tf' is not defined

In [11]:
# http://localhost:8501/v1/models/complaints
# http://35.216.48.148:8501/v1/models/complaints
HOST = "localhost"
PORT = 8501
MODEL_NAME = "complaints"
VERSION = 2
VERB = "predict"  # predict(기본), 서명 설정 시 classify, regress 가능
url = f"http://{HOST}:{PORT}/v1/models/{MODEL_NAME}/versions/{VERSION}:{VERB}"
print(url)
# ex_dec = base64.urlsafe_b64encode(example.SerializeToString()).decode('utf-8')
headers = {"content-type": "application/json"}
payload = {  # TFServing REST API 데이터는 JSON으로 주고 받는다.
    "signature_name": "serving_default",  # optional: serving_default 자체가 기본값.
    "instances": ["sample"]
}

# data = json.dumps(payload)

http://localhost:8501/v1/models/complaints/versions/2:predict


In [12]:
response = requests.post(url, json=json.dumps(payload))
# response = requests.post(url, headers=headers, data=data)
print(response.json())

{'error': 'JSON Value: "{\\"signature_name\\": \\"serving_default\\", \\"instances\\": [\\"sample\\"]}" Is not object'}


### gRPC
channel: 서버-호스트 간 gRPC 연결. 통신의 최초 전제.
stub: 사용 가능 메서드를 서버로부터 복사해 로컬에 저장한 객체.

In [ ]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2, prediction_service_pb2_grpc